In [1]:
import formulas
import utilities
import pandas as pd
import os
import re


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'

    formulas_input_df = pd.read_csv(formula_file_path)
    sort_orders_df = pd.read_csv(order_file_path)
    formulas_df = formulas.get_formulas_df(formulas_input_df, sort_orders_df)
    formula_sheet_list = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    
    utilities.delete_directory_if_it_exists(docs_dir)
    formulas.create_formulas_content(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    formulas.create_calculus_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)
    formulas.create_financial_summary(formulas_df, formula_sheet_list, 
                            sort_orders_df, docs_dir)


In [2]:
import os
import formulas
import utilities
import pandas as pd
import topics

def get_topic_display_string(input_series, **kwarg):
    
    """ !!!!MAY NEED TO UPDATE - not sure if I need to pass topics_df!!!
    Returns a topic summary in markdown format
    Input series is a pandas series with Indices State, Subect code
    and topic.  **Kwarg must be called with parameter  = topics_df
    where topics_df contains fields State,Topic sub category 1, 
    Topic sub category 2, Subject code and Topic
    """
    
    file_name = input_series['Topic'].lower().replace(' ', '_') + '.ipynb'
    file_path = kwarg['topics_dir'] + os.path.sep + file_name
    
    if os.path.isfile(file_path):
    
        if input_series['Topic sub category 2'].upper() == 'BY YEAR':
            tags = input_series['Subject code']
        elif (input_series['Topic sub category 2'].upper() == 
              'BY YEAR CUMULATIVE'):
            tags = topics.previous_subject_codes(
                kwarg['sort_orders_df'], input_series['Subject code'])
        else:
            tags = []
        
        output_str = utilities.filtered_notebook_md_string(
            input_notebook_path=file_path, include_tags=tags, 
            remove_input_tags=['hide_code'])
                                          
    return(output_str)


def create_topics_content(topics_df, sort_orders_df, docs_dir, topics_dir):
    """Creates markdown files containing topic tables as input for static 
    web page creation via Hugo
    """

    dirs_df = (
        topics_df[['State', 'Topic sub category 1', 
                             'Topic sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        topics_df[['State', 'Topic sub category 1', 
                             'Topic sub category 2',
                             'Subject code', 'Topic']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)

    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=get_topic_display_string, 
                           sort_orders_df=sort_orders_df,
                           topics_df=topics_df,
                           topics_dir=topics_dir)




if __name__=='__main__':
    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    
    sort_orders_df = pd.read_csv(order_file_path)
    topics_df = topics.get_topics_df(sort_orders_df)
    topics_dir = website_creator_dir + os.path.sep + 'topic_summaries'
    create_topics_content(topics_df, sort_orders_df, docs_dir, topics_dir)


In [8]:
topics_df

,State,Topic sub category 1,Topic sub category 2,Subject code,Topic
41,NSW,Topics,By year,Year 9,Indices
42,NSW,Topics,By year,Year 11 Adv,Calculus
43,NSW,Topics,By year,Year 12 Adv,Financial mathematics
44,NSW,Topics,By year,Year 12 Adv,Calculus
49,NSW,Topics,By year cumulative,Year 9,Indices
50,NSW,Topics,By year cumulative,Year 11 Adv,Indices
51,NSW,Topics,By year cumulative,Year 11 Adv,Calculus
52,NSW,Topics,By year cumulative,Year 12 Adv,Indices
53,NSW,Topics,By year cumulative,Year 12 Adv,Financial mathematics
54,NSW,Topics,By year cumulative,Year 12 Adv,Calculus


In [9]:
'By year cumulative'.upper()

'BY YEAR CUMULATIVE'

In [1]:
import os
import topics
import utilities
import pandas as pd
import numpy as np

In [7]:
subject_code = 'Year 12 Adv'




display(topics.previous_subject_codes(sort_orders_df, 'Year 11 Adv'))

['Year 9', 'Year 11 Adv']

In [18]:
website_creator_dir = os.getcwd()
data = np.array(['NSW', 'Topics', 'By year cumulative', 'Year 12 Ext 1', 'Financial mathematics'])
input_series = pd.Series(data, index=['State', 'Topic sub category 1', 'Topic sub category 2', 'Subject code', 'Topic'])

# file_name = input_series['Topic'].lower().replace(' ', '_') + '.ipynb'

# file_path  =   os.path.sep.join([website_creator_dir, 'topic_summaries', file_name])

# if os.path.isfile(file_path):

#     if input_series['Topic sub category 2'].upper() == 'BY YEAR':
#         tags = input_series['Subject code']
#     elif input_series['Topic sub category 2'].upper() == 'BY YEAR CUMULATIVE':
#         tags = topics.previous_subject_codes(
#             sort_orders_df, input_series['Subject code'])
#     else:
#         tags = []
    
#     output_str = utilities.filtered_notebook_md_string(
#         input_notebook_path=file_path, include_tags=tags, 
#         remove_input_tags=['hide_code'])
    
#     display(output_str)

'# Financial mathematics\n\n## Compounding interest\n\n\n$\\displaystyle A = P(1+r)^n$\n\n\n## Arithmetic sequence and series\n\nAn **arithmetic sequence** is a sequence of numbers such that the difference from any succeeding term to its preceding term remains constant throughout the sequence\n\nAn **arithmetic series** is sum of the members of a finite arithmetic sequence\n'